### Import

In [2]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms


### Create Fully Connected Network

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [4]:
model = NN(784, 10)

In [5]:
x = torch.randn(64, 784)
x

tensor([[ 1.3673,  0.6008,  1.3140,  ..., -0.2486, -0.7746,  0.9668],
        [-0.4707,  1.4091,  0.3496,  ...,  0.3469, -1.0280,  1.4500],
        [-0.6202, -1.3229,  0.4583,  ...,  0.0636, -1.5339, -0.1134],
        ...,
        [ 2.0049, -0.5189,  0.3293,  ...,  1.4918, -1.6590, -0.1176],
        [ 1.2488,  0.0712,  0.0936,  ...,  0.3931,  2.0859, -2.0725],
        [-0.3703,  1.4006,  0.4256,  ..., -1.1941, -0.9300, -2.3218]])

In [6]:
model(x).shape

torch.Size([64, 10])

### Set Device

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Hyperparameters

In [8]:
input_size = 784
num_classes = 10 
learning_rate = 0.001
batch_size = 64 
num_epochs = 1

### Load Data

In [9]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Initialize Network

In [10]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

### Loss and Optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train

In [12]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Get to correct shape
        data = data.reshape(data.shape[0],-1)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient decent or adam step
        optimizer.step() # update weight
        

In [13]:
data.shape

torch.Size([32, 784])

### Evaluation

In [14]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")  
          
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
        
    model.train()

In [15]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 55837 / 60000 with accuracy 93.06
Checking accuracy on test data
Got 9307 / 10000 with accuracy 93.07
